In [1]:
try:
    from qiskit.primitives import Estimator  # 使用新的StatevectorEstimator替代
    from qiskit.quantum_info import SparsePauliOp
    from qiskit.circuit.library import EfficientSU2, TwoLocal
    QISKIT_AVAILABLE = True
except ImportError:
    QISKIT_AVAILABLE = False

In [2]:
QISKIT_AVAILABLE

False

In [4]:
from qiskit.primitives import StatevectorEstimator  # 使用新的StatevectorEstimator替代

In [ ]:
estimator = StatevectorEstimator()
job = estimator.run([(ansatz, hamiltonian, params)])

In [5]:
#!/usr/bin/env python3
"""
Qiskit StatevectorEstimator 简单示例

这个示例展示了如何使用 StatevectorEstimator 计算量子电路的期望值。
"""

import numpy as np
from qiskit.primitives import StatevectorEstimator
from qiskit.quantum_info import SparsePauliOp
from qiskit.circuit.library import TwoLocal

# 1. 创建 ansatz (参数化量子电路)
# 使用 TwoLocal 创建一个简单的参数化电路
# 2个量子比特，使用RY和RZ旋转门，线性纠缠
ansatz = TwoLocal(
    num_qubits=2,
    rotation_blocks=['ry', 'rz'],
    entanglement_blocks='cx',
    entanglement='linear',
    reps=1,
    insert_barriers=True
)

# 2. 创建 hamiltonian (观测量/哈密顿量)
# 创建一个简单的哈密顿量：Z⊗Z + X⊗Y
hamiltonian = SparsePauliOp.from_list([
    ("ZZ", 1.0),  # Z⊗Z 项，系数为 1.0
    ("XY", 0.5)   # X⊗Y 项，系数为 0.5
])

# 3. 创建 params (参数值)
# 为 ansatz 电路创建随机参数值
# ansatz.num_parameters 返回电路需要的参数数量
params = np.random.uniform(0, 2*np.pi, ansatz.num_parameters)

print("=== Qiskit StatevectorEstimator 示例 ===")
print(f"Ansatz 电路参数数量: {ansatz.num_parameters}")
print(f"随机参数值: {params}")
print(f"哈密顿量: {hamiltonian}")

# 4. 创建 StatevectorEstimator 实例
estimator = StatevectorEstimator()

# 5. 运行计算
# 使用元组列表格式：(circuit, observable, parameter_values)
job = estimator.run([(ansatz, hamiltonian, params)])

# 6. 获取结果
result = job.result()

# 7. 访问期望值
# 注意：在 Qiskit 1.4.4 中，正确的访问方式是 result[0].data.evs
energy = result[0].data.evs

print(f"计算得到的期望值: {energy}")

# 8. 可选：可视化电路
print("\nAnsatz 电路结构:")
print(ansatz.draw(output='text'))

# 9. 可选：尝试不同的参数值
print("\n=== 尝试不同的参数值 ===")
# 使用全零参数
zero_params = np.zeros(ansatz.num_parameters)
job_zero = estimator.run([(ansatz, hamiltonian, zero_params)])
result_zero = job_zero.result()
energy_zero = result_zero[0].data.evs
print(f"全零参数的期望值: {energy_zero}")

# 使用随机参数
random_params = np.random.uniform(0, 2*np.pi, ansatz.num_parameters)
job_random = estimator.run([(ansatz, hamiltonian, random_params)])
result_random = job_random.result()
energy_random = result_random[0].data.evs
print(f"随机参数的期望值: {energy_random}")


C:\Users\Administrator\AppData\Local\Temp\ipykernel_21216\3757545227.py:16: DeprecationWarning: The class ``qiskit.circuit.library.n_local.two_local.TwoLocal`` is deprecated as of Qiskit 2.1. It will be removed in Qiskit 3.0. Use the function qiskit.circuit.library.n_local instead.
  ansatz = TwoLocal(


=== Qiskit StatevectorEstimator 示例 ===
Ansatz 电路参数数量: 8
随机参数值: [0.50624868 2.98087626 3.79320762 1.42568423 1.1116505  1.35160416
 6.22442126 3.14082392]
哈密顿量: SparsePauliOp(['ZZ', 'XY'],
              coeffs=[1. +0.j, 0.5+0.j])
计算得到的期望值: -0.41423647126510554

Ansatz 电路结构:
     ┌────────────────────────────────────────────────────┐
q_0: ┤0                                                   ├
     │  TwoLocal(θ[0],θ[1],θ[2],θ[3],θ[4],θ[5],θ[6],θ[7]) │
q_1: ┤1                                                   ├
     └────────────────────────────────────────────────────┘

=== 尝试不同的参数值 ===
全零参数的期望值: 1.0
随机参数的期望值: -0.06056514222189463


In [10]:
def build_hamiltonian(problem_config, n_qubits: int) -> hamiltonians.SymbolicHamiltonian:
    """
    使用SymbolicHamiltonian构建Qibo哈密顿量，保证公平、高效和简洁。
    """

    
    from qibo import symbols  # 导入Qibo符号模块
    
    j_coupling = problem_config.get("j_coupling", 1.0)
    h_field = problem_config.get("h_field", 1.0)
    
    # 使用Qibo符号构建哈密顿量
    ham_expr = 0
    
    # 相互作用项: -J * sum(Z_i Z_{i+1})
    for i in range(n_qubits):
        Z_i = symbols.Z(i)
        Z_ip1 = symbols.Z((i + 1) % n_qubits)
        ham_expr += -j_coupling * Z_i * Z_ip1
    
    # 横向场项: -h * sum(X_i)
    for i in range(n_qubits):
        X_i = symbols.X(i)
        ham_expr += -h_field * X_i
    
    return hamiltonians.SymbolicHamiltonian(ham_expr)

In [8]:
from qibo import symbols 
from qibo import hamiltonians

In [11]:
build_hamiltonian({"j_coupling": 1.0, "h_field": 1.0}, 2)

[Qibo 0.2.21|INFO|2025-10-16 20:43:51]: Using qibojit (numba) backend on /CPU:0
